In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
mnist_tmaxs = [2**exponent for exponent in range(1, 6)]
mnist_error = 100 - np.array([89.68999981880188, 98.10999631881714, 99.04999732971191, 99.1599977016449, 99.24999475479126])

cifar_tmaxs = [2**exponent for exponent in range(3, 7)]
cifar_error = 100 - np.array([83.30999612808228, 91.3599967956543, 92.14999675750732, 92.25999712944031])

imagenet_tmaxs = [2**exponent for exponent in range(4, 7)]
imagenet_error = 100 - np.array([54.44, 64.44, 67.24])
imagenet_times = [146, 290, 578]

In [ ]:
plt.scatter(mnist_tmaxs, mnist_error)
plt.plot(mnist_tmaxs, mnist_error, label='MNIST')
plt.scatter(cifar_tmaxs, cifar_error)
plt.plot(cifar_tmaxs, cifar_error, label="CIFAR10")
plt.scatter(imagenet_tmaxs, imagenet_error)
plt.plot(imagenet_tmaxs, imagenet_error, label="ImageNet")
plt.xscale('log')
plt.xlim(right=100)
plt.ylabel("Accuracy error [%]")
plt.xlabel(r"$T_{\mathrm{max}}$ per layer")
plt.legend()
plt.grid()
plt.tight_layout()
plt.savefig("simulation_accs.png")

In [ ]:
mnist_tmaxs = [2**exponent for exponent in range(1, 6)]
mnist_error = 100 - np.array([89.68999981880188, 98.10999631881714, 99.04999732971191, 99.1599977016449, 99.24999475479126])
mnist_ops = np.array([49, 66, 101, 171, 310])*1e-3

plt.plot(mnist_ops, mnist_error, linestyle='--')
plt.scatter(mnist_ops, mnist_error, label='MNIST SNN')
plt.scatter(45.168, 100-99.22999739646912, marker='x', s=100, color='C0', label='MNIST ANN')
plt.xscale('log')
# plt.xlim(right=100)
plt.ylabel("Accuracy error [%]")
plt.xlabel(r"MOps / frame")
plt.legend()
plt.grid()
plt.tight_layout()
plt.savefig("simulation_accs.png")

In [ ]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

df = pd.DataFrame({
    'Dataset' : ['MNIST'] * 5,
    'Text' : [f"Tmax {2**exponent}" for exponent in range(1, 6)],
    'Accuracy error [%]': 100 - np.array([89.68999981880188, 98.10999631881714, 99.04999732971191, 99.1599977016449, 99.24999475479126]),
    'Number of operations [MOps]' : np.array([49, 66, 101, 171, 310])*1e-3
})

df2 = pd.DataFrame({
    'Dataset': ['MNIST'],
    'Text': 'ANN',
    'Accuracy error [%]': 100-99.22999739646912,
    'Number of operations [MOps]': 45.168,
})

fig1 = px.line(df, x="Number of operations [MOps]", y="Accuracy error [%]", color="Dataset", text="Text")
fig1.update_traces(textposition="top right")

fig2 = px.scatter(df2, x="Number of operations [MOps]", y="Accuracy error [%]", color="Dataset", text="Text")
fig2.update_traces(textposition="top right")
fig = go.Figure(data=fig1.data + fig2.data, )
fig.update_xaxes(type="log")
fig.show()
fig.write_image('n_ops.png')